*This fun, step by step, project "A/B Testing for ShoeFly.com" is at the end of the chapter of Codecademy's chapter on Python Aggregates in Pandas in Python Pandas for Data Science, part of Data Science Foundations II, which is overrall part of the career path Data Scientist Machine Learning.*

*(It's a big course).*

*While this could be done in the interactive Codecademy window, I decided to copy it here into Jupyter, just so there's actually a record that I did the dang thing.*

*It consolidates and tests one's knowledge of carrying out aggregates on dataframes in Pandas.*

*Enjoy.*

# A/B Testing for ShoeFly.com

Our favorite online shoe store, ShoeFly.com is performing an A/B Test. They have two different versions of an ad, which they have placed in emails, as well as in banner ads on Facebook, Twitter, and Google. They want to know how the two ads are performing on each of the different platforms on each day of the week. Help them analyze the data using aggregate measures.

## Analyzing Ad Sources

1. Examine the first few rows of `ad_clicks.`

In [1]:
import pandas as pd

ad_clicks = pd.read_csv('ad_clicks.csv')
ad_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B


2. Your manager wants to know which ad platform is getting you the most views.

    How many views (i.e., rows of the table) came from each `utm_source?`

In [4]:
ads_by_source = ad_clicks.groupby('utm_source').user_id.count().reset_index()
ads_by_source = ads_by_source.rename(columns={'user_id': 'counts'})
print(ads_by_source)

  utm_source  counts
0      email     255
1   facebook     504
2     google     680
3    twitter     215


3. If the column `ad_click_timestamp` is not null, then someone actually clicked on the ad that was displayed.

    Create a new column called `is_click`, which is True if `ad_click_timestamp` is not null and `False` otherwise.

In [5]:
ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()
ad_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False


4. We want to know the percent of people who clicked on ads from each `utm_source`.

    Start by grouping by `utm_source` and `is_click` and counting the number of `user_id`‘s in each of those groups. Save your answer to the variable `clicks_by_source`.

In [6]:
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()
print(clicks_by_source)

  utm_source  is_click  user_id
0      email     False      175
1      email      True       80
2   facebook     False      324
3   facebook      True      180
4     google     False      441
5     google      True      239
6    twitter     False      149
7    twitter      True       66


5. Now let’s pivot the data so that the columns are `is_click` (either `True` or `False`), the index is `utm_source`, and the values are `user_id`.

    Save your results to the variable `clicks_pivot`.

In [9]:
clicks_pivot = clicks_by_source.pivot(columns='is_click',
                                      index='utm_source',
                                      values='user_id').reset_index()
print(clicks_pivot)

is_click utm_source  False  True
0             email    175    80
1          facebook    324   180
2            google    441   239
3           twitter    149    66


6. Create a new column in `clicks_pivot` called `percent_clicked` which is equal to the percent of users who clicked on the ad from each `utm_source`.

    Was there a difference in click rates for each source?

In [12]:
clicks_pivot['percent_clicked'] = clicks_pivot[True]/(clicks_pivot[True] + clicks_pivot[False])
print(clicks_pivot)

is_click utm_source  False  True  percent_clicked
0             email    175    80         0.313725
1          facebook    324   180         0.357143
2            google    441   239         0.351471
3           twitter    149    66         0.306977


**My comment: Yes there was a difference in click rates**

## Analyzing an A/B Test

7. The column `experimental_group` tells us whether the user was shown Ad A or Ad B.

    Were approximately the same number of people shown both ads?

In [13]:
by_experiment = ad_clicks.groupby('experimental_group').user_id.count().reset_index()
print(by_experiment)

  experimental_group  user_id
0                  A      827
1                  B      827


**My comment: Same number of people were shown both ads**

8. Using the column `is_click` that we defined earlier, check to see if a greater percentage of users clicked on Ad A or Ad B.

In [18]:
#Group table by experiment and is_click
clicks_by_experiment = ad_clicks.groupby(['experimental_group', 'is_click']).user_id.count().reset_index()

#Pivot table 
clicks_by_experiment_pivot = clicks_by_experiment.pivot(columns='is_click',
                                                       index='experimental_group',
                                                       values='user_id').reset_index()

#Calculate percent clicked and add it to new column
clicks_by_experiment_pivot['percent_clicked'] = clicks_by_experiment_pivot[True]/(clicks_by_experiment_pivot[True] + clicks_by_experiment_pivot[False])

#Print final table
print(clicks_by_experiment_pivot)

is_click experimental_group  False  True  percent_clicked
0                         A    517   310         0.374849
1                         B    572   255         0.308343


**My comment: Greater percent of people clicked on Ad A.**

9. The Product Manager for the A/B test thinks that the clicks might have changed by day of the week.

    Start by creating two DataFrames: `a_clicks` and `b_clicks`, which contain only the results for `A` group and `B` group, respectively.

10. For each group (`a_clicks` and `b_clicks`), calculate the percent of users who clicked on the ad by `day`.

11. Compare the results for A and B. What happened over the course of the week?

    Do you recommend that your company use Ad `A` or Ad `B`?